In [1]:
import json, jsonschema, jwt, datetime, requests

## Utility functions

In [2]:
def mint_merit_did(uuid):
    return f'did:merit:{uuid}'

In [3]:
def jwt_serialization(credential, secret_key, audience):
    payload = {
        "iss": credential["issuer"],  # Issuer
        "aud": audience,  # Audience
        "exp": datetime.datetime.utcnow() + datetime.timedelta(days=1),  # Expiration time
        "nbf": datetime.datetime.utcnow(),  # Not before
        "iat": datetime.datetime.utcnow(),  # Issued at
    }
    if "VerifiableCredential" in credential["type"]:
        payload["sub"] = credential["credentialSubject"]["id"]
        payload["vc"] = credential
    elif "VerifiablePresentation" in credential["type"]:
        payload["vp"] = credential
    return jwt.encode(payload, secret_key, algorithm='HS256')

## Merit credential

In [4]:
merit = json.load(open("test_merit.json", "r"))
merit

{'id': 'dc30880d-2d4b-447c-905e-b79075aea95b',
 'acceptedAt': '2023-07-17T08:52:19.489566Z',
 'active': True,
 'activenessFailures': None,
 'authorizedAt': '2023-07-17T08:52:15.960677Z',
 'completed': True,
 'completenessFailures': None,
 'createdAt': '2023-07-17T15:52:04.3018Z',
 'description': 'Template for admin merit',
 'fields': [{'canShare': False,
   'description': "Use this to store recipient's first name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'First Name',
   'templateFieldID': '1b397a27-14a0-4d43-a36f-6006611f4073',
   'templateFieldLineage': ['3dc3f769-70cc-4e7e-81cc-ed103f9eb601'],
   'updatedAt': '2023-07-17T15:52:19.496539Z',
   'validationErrors': None,
   'value': 'Hank'},
  {'canShare': False,
   'description': "Use this to store recipient's last name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'Last Name',
   

## Verifiable credential

In [5]:
verifiable_credential = {}

### Contexts

In [6]:
verifiable_credential["@context"] = [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1",
    "https://schema.org/"
]

### Identifiers

In [7]:
id = mint_merit_did(merit["id"])
issuer = mint_merit_did(merit["issuer"]["id"])
subject = mint_merit_did(merit["recipient"]["id"])

verifiable_credential["issuer"] = issuer
verifiable_credential["id"] = id

### Types

In [8]:
verifiable_credential["type"] = ["VerifiableCredential", merit["name"]]

### Credential Subject

In [9]:
verifiable_credential["credentialSubject"] = { 
    "id": subject,
    "merit": { field["name"]: field["value"] for field in merit["fields"] },
}

### Issuance date

In [10]:
verifiable_credential["issuanceDate"] = merit["createdAt"]

### Credential Status

In [11]:
verifiable_credential["credentialStatus"] = {
    "id": f'https://vc.merit.com/status/{requests.utils.quote(id)}',
    "type": "MeritCredentialStatusList2024",
    "statusPurpose": merit['state']['name'],
    "occurredAt": merit['state']['occurredAt']
}

## JSON Schema validation

In [12]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiableCredential"
            }
        },
        "issuer": {
            "type": "string"
        },
        "issuanceDate": {
            "type": "string",
            "format": "date-time"
        },
        "credentialSubject": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                }
            },
            "required": ["id"]
        },
        "credentialStatus": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                },
                "type": {
                    "type": "string"
                },
                "statusPurpose": {
                    "type": "string"
                },
                "occurredAt": {
                    "type": "string",
                    "format": "date-time"
                }
            },
            "required": ["id"]
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "type", "issuer", "issuanceDate", "credentialSubject"]#, "proof"]
}

In [13]:
try:
    jsonschema.validate(instance=verifiable_credential, schema=schema)
    print("credential is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("credential is not valid:", e)

credential is valid.


## JSON Web Token (JWT) serialization

In [14]:
vc_jwt_token = jwt_serialization(verifiable_credential, "secret", "https://verifier.example.gov")

In [15]:
decoded_payload = jwt.decode(vc_jwt_token, "secret", algorithms='HS256', audience='https://verifier.example.gov')
decoded_payload['vc']

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'did:merit:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://vc.merit.com/status/did%3Amerit%3Adc30880

## Geofencing

In [16]:
verifiable_credential["credentialSubject"]["eligibleRegion"] = {
    "@type": "GeoCircle",
    "geoMidpoint": {
        "@type": "GeoCoordinates",
        "latitude": "38.88508",
        "longitude": "-77.02295"
    },
    "geoRadius": "50"
}

In [17]:
verifiable_credential

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'did:merit:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'},
  'eligibleRegion': {'@type': 'GeoCircle',
   'geoMidpoint': {'@type': 'GeoCoordinates',
    'latitude': '38.88508',
    'lon

In [18]:
vc_jwt_token = jwt_serialization(verifiable_credential, "secret", "https://verifier.example.gov")

## Geolocation

In [19]:
geolocation_vc = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
    'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075dec95b',
    'type': ['VerifiableCredential', 'RealtimeGeolocation'],
    'credentialSubject' : {
        'id': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
        'geo': {
            'type': 'GeoCoordinates',
            'latitude': '38.88460',
            'longitude': '-77.02273'
        }
    },
    'issuanceDate': datetime.datetime.utcnow().isoformat() + 'Z'
}

In [20]:
geolocation_vc_jwt_token = jwt_serialization(geolocation_vc, "secret", "https://verifier.example.gov")

In [21]:
verifiable_presentation = {
    '@context': [
        'https://www.w3.org/2018/credentials/v1', 
        'https://www.w3.org/2018/credentials/examples/v1', 
        'https://schema.org/'
    ],
    'issuer': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
    'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075dec95b',
    'type': ['VerifiablePresentation'],
    'verifiableCredential': [
        vc_jwt_token,
        geolocation_vc_jwt_token
    ]
}

In [22]:
verifiable_presentation

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075dec95b',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': ['eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJkaWQ6bWVyaXQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiYXVkIjoiaHR0cHM6Ly92ZXJpZmllci5leGFtcGxlLmdvdiIsImV4cCI6MTY5NDkxMzU4MywibmJmIjoxNjk0ODI3MTgzLCJpYXQiOjE2OTQ4MjcxODMsInN1YiI6ImRpZDptZXJpdDoxOTZhOGM4Zi02ZmQ5LTQ1NmItODE5My00MTU5YjhkYzNhYjYiLCJ2YyI6eyJAY29udGV4dCI6WyJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy92MSIsImh0dHBzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL2V4YW1wbGVzL3YxIiwiaHR0cHM6Ly9zY2hlbWEub3JnLyJdLCJpc3N1ZXIiOiJkaWQ6bWVyaXQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwiaWQiOiJkaWQ6bWVyaXQ6ZGMzMDg4MGQtMmQ0Yi00NDdjLTkwNWUtYjc5MDc1YWVhOTViIiwidHlwZSI6WyJWZXJpZmlhYmxlQ3JlZGVudGlhbCIsIkFkbWluIG1lcm

In [23]:
geoaware_vp_jwt_token = jwt_serialization(verifiable_presentation, "secret", "https://verifier.example.gov")

In [25]:
decoded_payload = jwt.decode(geoaware_vp_jwt_token, "secret", algorithms='HS256', audience='https://verifier.example.gov')
decoded_vp = decoded_payload['vp']
decoded_vp["verifiableCredential"] = [ 
    jwt.decode(payload, "secret", algorithms='HS256', audience='https://verifier.example.gov')["vc"] for payload in decoded_vp["verifiableCredential"] 
]
decoded_vp

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://schema.org/'],
 'issuer': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
 'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075dec95b',
 'type': ['VerifiablePresentation'],
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v1',
    'https://www.w3.org/2018/credentials/examples/v1',
    'https://schema.org/'],
   'issuer': 'did:merit:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
   'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075aea95b',
   'type': ['VerifiableCredential', 'Admin merit'],
   'credentialSubject': {'id': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
    'merit': {'First Name': 'Hank',
     'Last Name': 'Lindgren',
     'Email': 'oleg+user2@gomerits.com',
     'Issuing Org Name': 'Merit International, Inc.',
     'Org UUID': 'user2',
     'Admin Phone Number': '6317070926',
     'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3